<a href="https://colab.research.google.com/github/tcglarry/geo/blob/master/stage_2_reproduce_Unet_colab_correct_ch32_NRMSE_loss.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# -*- coding: utf-8 -*-
"""ali_build_model_stage_2_MobileNetV2_aug_1000_dense.ipynb

Automatically generated by Colaboratory.

Original file is located at
    https://colab.research.google.com/drive/1wSVEmcoWzRm-7B6UGkHB8ysfMDRT_xK5
"""

# Install a Drive FUSE wrapper.
# https://github.com/astrada/google-drive-ocamlfuse
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse

E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package google-drive-ocamlfuse.
(Reading database ... 110842 files and directories currently installed.)
Preparing to unpack .../google-drive-ocamlfuse_0.7.1-0ubuntu3~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.1-0ubuntu3~ubuntu18.04.1) ...
Setting up google-drive-ocamlfuse (0.7.1-0ubuntu3~ubuntu18.04.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
src=  'drive/My Drive/geo/'

In [0]:

import numpy as np
import matplotlib.pyplot as plt
import keras.backend as K
import tensorflow as tf
import subprocess
import os
import pickle
from keras.layers import Input, Conv2D, MaxPooling2D, merge, Lambda,UpSampling2D, concatenate, Conv2DTranspose
from keras.models import Model, load_model
import pandas as pd
import sklearn 
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import normalize
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from keras import regularizers

In [6]:
data= np.load(src+'train_data.npy')
label = np.load(src+'train_labels.npy')
print (np.mean(label))
label = label[:,:,:,np.newaxis]
print (data.shape)
print (label.shape)

5.66184075511905
(4200, 21, 21, 20)
(4200, 20, 20, 1)


In [0]:

data_mean = np.mean(data)
data_std = np.std(data)
label_max = np.max(label)
label_min = np.min(label)

train_data = (data - data_mean)/data_std
#label_data = (label-label_min)/(label_max - label_min)
label_data = np.log(label)

In [0]:
train_x,val_x,train_y,val_y = train_test_split(train_data,label_data,test_size=0.2, shuffle=True)

In [0]:
def conv_block(ch, activation= 'relu', padding='same',kernel_regularizer=regularizers.l2(0.01)):       
    return Conv2D(ch,(3,3),activation= activation, padding =padding )


In [0]:
#def build_model_1(IMAGE_HEIGHT=IMAGE_HEIGHT,IMAGE_WIDTH=IMAGE_WIDTH,ch=ch):
def build_model_1(ch=32):
    #inputs = Input((IMAGE_HEIGHT,IMAGE_WIDTH,ch))
    inputs = Input((21,21,20))

    conv0 = Conv2D(32,(2,2),padding='valid')(inputs)
    print ('conv0',conv0.get_shape())
    conv1 =  conv_block(ch)(conv0)
    conv1 = conv_block(ch)(conv1)
    pool1 = MaxPooling2D(pool_size=(2,2))(conv1)
    print ('pool1',pool1.get_shape())
    
    conv2 =  conv_block(ch*2)(pool1)
    conv2 = conv_block(ch*2)(conv2)
    pool2 = MaxPooling2D(pool_size=(2,2))(conv2)
    print ('poo12',pool2.get_shape())
    
    conv_test = Conv2D(64,(2,2),padding='valid')(pool2)
    print ('conv_test',conv_test.get_shape())
    
    
    
    conv3 =  conv_block(ch*4)(conv_test)
    conv3 = conv_block(ch*4)(conv3)
    pool3 = MaxPooling2D(pool_size=(2,2))(conv3)
    print ('poo13',pool3.get_shape())
    
    conv4 =  conv_block(ch*8)(pool3)
    conv4 = conv_block(ch*8)(conv4)
    pool4 = MaxPooling2D(pool_size=(2,2))(conv4)
    print ('poo14',pool4.get_shape())
    
    conv5 =  conv_block(ch*16)(pool4)
    conv5 = conv_block(ch*16)(conv5)

    
    up6 = concatenate([UpSampling2D(size=(2,2))(conv5), conv4], axis=3)
    conv6 = conv_block(ch*8)(up6)
    conv6 = conv_block(ch*8)(conv6)
    
    up7 = concatenate ([UpSampling2D(size=(2,2))(conv6), conv3],  axis=3)
    conv7 = conv_block(ch*4)(up7)
    conv7 = conv_block(ch*4)(conv7)
    print ('conv7',conv7.get_shape())
    conv7 = Conv2DTranspose(ch*4,(2,2),padding='valid')(conv7)
    print ('conv7',conv7.get_shape())
    
    up8 = concatenate([UpSampling2D(size=(2,2))(conv7), conv2], axis=3)
    conv8 = conv_block(ch*2)(up8)
    conv8 = conv_block(ch*2)(conv8)
    
    up9 = concatenate([UpSampling2D(size=(2,2))(conv8), conv1],  axis=3)
    conv9 = conv_block(ch)(up9)
    conv9 = conv_block(ch)(conv9)
    
    
    
    conv10 = Conv2D(1,(1,1))(conv9)
    
  
    model = Model(inputs= inputs, outputs=conv10)
    
    return model
    
    

In [28]:
model = build_model_1()
model.summary()

conv0 (?, 20, 20, 32)
pool1 (?, 10, 10, 32)
poo12 (?, 5, 5, 64)
conv_test (?, 4, 4, 64)
poo13 (?, 2, 2, 128)
poo14 (?, 1, 1, 256)
conv7 (?, 4, 4, 128)
conv7 (?, ?, ?, 128)
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 21, 21, 20)   0                                            
__________________________________________________________________________________________________
conv2d_22 (Conv2D)              (None, 20, 20, 32)   2592        input_2[0][0]                    
__________________________________________________________________________________________________
conv2d_23 (Conv2D)              (None, 20, 20, 32)   9248        conv2d_22[0][0]                  
__________________________________________________________________________________________________
conv2d_24 (Conv2D)              (Non

In [0]:



def gram_matrix(x):
  #features = K.batch_flatten(K.permute_dimensions(x[0], (2, 0, 1)))
  x = K.batch_flatten(x)
  gram = K.dot(x, K.transpose(x))
  print(gram)
  #return K.expand_dims(gram, axis=0)
  return gram
  


def my_loss(y_true,y_pred):
  loss_1 = K.mean(K.square(y_pred - y_true), axis=-1)
  loss_2= K.categorical_crossentropy(y_true, y_pred)
  #diff_2 = K.mean(K.square(gram_matrix(y_pred) - gram_matrix(y_true)), axis=-1)
  
  total_loss = loss_1 * 1. + 5. * loss_2
  return total_loss
  



In [0]:
model.compile (optimizer='Adam', loss = 'mse', metrics =['mae'])

In [15]:

# train the model on the new data for a few epochs

use_model= False
if use_model:
  
    

  if os.path.isfile(src+'water_first_try_2.h5'):

    model = load_model(src+'water_first_try_2.h5')
    print ('model laoded')
  else:
    print ('model not exist')
      
else:
  print ('no use model')
  


no use model


In [0]:
del model

In [30]:

checkpoint = ModelCheckpoint(filepath=src+'stage2_reproduce_water_first_try_2_prepro_ch32_check_nrmse.h5', monitor = 'val_loss', save_best_only=True, mode= 'auto')
earlystop = EarlyStopping(patience=20)
lr_reduce = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=15, verbose=1, mode='auto', min_lr=0.00001)
callback_list = [checkpoint, earlystop]

model.fit(x=train_x, y=train_y, batch_size=64, epochs=300, callbacks = callback_list, verbose=1, validation_split=0.2,  shuffle=True)


Train on 2688 samples, validate on 672 samples
Epoch 1/300
2688/2688 [==============================] - 6s 2ms/step - loss: 1.3310 - mean_absolute_error: 0.8913 - val_loss: 0.7983 - val_mean_absolute_error: 0.7127
Epoch 2/300
2688/2688 [==============================] - 4s 1ms/step - loss: 0.6973 - mean_absolute_error: 0.6632 - val_loss: 0.5174 - val_mean_absolute_error: 0.5660
Epoch 3/300
2688/2688 [==============================] - 4s 1ms/step - loss: 0.4788 - mean_absolute_error: 0.5431 - val_loss: 0.4058 - val_mean_absolute_error: 0.5008
Epoch 4/300
2688/2688 [==============================] - 3s 1ms/step - loss: 0.3418 - mean_absolute_error: 0.4554 - val_loss: 0.2964 - val_mean_absolute_error: 0.4235
Epoch 5/300
2688/2688 [==============================] - 3s 1ms/step - loss: 0.2832 - mean_absolute_error: 0.4143 - val_loss: 0.2521 - val_mean_absolute_error: 0.3926
Epoch 6/300
2688/2688 [==============================] - 4s 1ms/step - loss: 0.2358 - mean_absolute_error: 0.3768 - va

In [34]:
error = model.evaluate(val_x,val_y)
print (error)

840/840 [==============================] - 0s 591us/step
[0.03607211040244216, 0.13424101244835626]


In [0]:
def nrmse(y_true,y_pred):
    
    return (np.sqrt(np.mean(np.square(y_true - y_pred))))/np.mean(y_pred)
def calculate_nrmse():
  y_pred = np.exp(model.predict(val_x))
  print (f'shape of y_pred {y_pred.shape}')
  y_true = np.exp(val_y)
  print (f'shape of y_true {y_true.shape}')
  
  loss =  nrmse(y_true,y_pred)
  
  print (loss)

  return loss

In [36]:
nrmse_loss = calculate_nrmse()

shape of y_pred (840, 20, 20, 1)
shape of y_true (840, 20, 20, 1)
0.2719575359504181


In [37]:
print (os.path.isfile(src+'test2.npy'))
test = np.load(src+'test2.npy')
test_data = (test - data_mean)/data_std
test_data.shape

True


(1800, 21, 21, 20)

In [49]:
prediction = model.predict(test_data)
prediction = np.exp(prediction)
pred = prediction.reshape(1800,-1)
print (pred.shape)


pred_df = pd.DataFrame(pred)
pred_df.keys()


(1800, 400)


RangeIndex(start=0, stop=400, step=1)

In [0]:
src_2 =  'drive/My Drive/geo/test_data/test_dataset/'

In [46]:
test_label_df = pd.read_csv(src_2+'solution.csv')
test_df = pd.read_csv(src+'upload_sample.csv')

x = test_label_df['ID'] != test_df['ID']
sum(x)


0

In [47]:
test_label_df = test_label_df.drop('ID', axis=1)
test_label_df.keys()

Index(['loc1', 'loc2', 'loc3', 'loc4', 'loc5', 'loc6', 'loc7', 'loc8', 'loc9',
       'loc10',
       ...
       'loc391', 'loc392', 'loc393', 'loc394', 'loc395', 'loc396', 'loc397',
       'loc398', 'loc399', 'loc400'],
      dtype='object', length=400)

In [48]:
test_label = test_label_df.values
test_label.shape

(1800, 400)

In [50]:
nrmse(test_label,pred)

0.2777077958843121

In [0]:
test_df = pd.read_csv(src+'upload_sample.csv')
col_names = test_df.keys().tolist()

In [0]:
final = pd.concat([test_df['ID'],pred_df],axis=1)
final.columns = col_names
final.to_csv(src+'stage_2_test_submit_0831_ch32.csv', index=False)
print ('submit file saved')

submit file saved


In [0]:
test_see_df = pd.read_csv(src+'test_submit_0831_ch32.csv')
test_see_df

,ID,loc1,loc2,loc3,loc4,loc5,loc6,loc7,loc8,loc9,...,loc391,loc392,loc393,loc394,loc395,loc396,loc397,loc398,loc399,loc400
0,F0000009,2.303276,2.633120,0.967853,0.711195,0.612320,0.606436,0.659745,0.903510,1.337734,...,14.198577,8.562691,2.988225,5.268833,4.511308,2.368932,3.126402,3.378817,4.378636,3.183275
1,F0000011,6.329229,5.513133,3.744405,2.303595,1.706632,2.405296,1.752342,1.457627,0.967599,...,2.244529,2.195623,1.610037,1.782069,1.366208,0.615595,0.960376,0.557508,0.523693,0.649285
2,F0000012,3.611720,5.277518,3.246185,1.668042,0.475114,0.388565,0.637970,0.502681,0.655798,...,1.773896,2.336798,1.996541,1.867982,1.414425,0.676193,0.828465,1.352033,1.516963,0.926687
3,F0000014,2.683136,2.873166,1.564418,0.715690,0.310166,0.841908,2.362045,2.305695,1.127642,...,2.207136,2.887102,8.945212,9.453363,10.373502,6.186830,5.693193,4.179819,2.414889,3.788618
4,F0000017,0.234505,0.538168,0.824665,1.241247,0.794359,0.691293,2.202758,3.200006,2.721208,...,6.542584,22.577510,32.344440,18.713957,15.682641,5.173859,2.250664,1.221537,5.205012,2.196197
5,F0000018,3.106924,4.724935,3.024786,2.114809,1.122492,0.463340,0.880873,1.015929,1.780746,...,2.085273,2.548799,0.884403,1.468000,3.873965,5.753904,2.968388,1.870800,4.490972,9.098993
6,F0000024,1.509835,1.381073,2.387015,1.938448,1.519532,1.164685,1.023207,1.416044,1.884219,...,0.844937,1.718213,3.039999,2.531460,2.925235,4.045678,2.195813,2.420788,4.041779,3.283170
7,F0000036,4.549304,4.280723,2.687711,7.107157,5.854353,6.881705,8.659466,5.750557,8.395898,...,8.959080,4.130376,4.660564,7.090211,10.861546,14.283480,18.413473,7.699151,7.086923,8.154278
8,F0000046,5.534216,9.097797,25.130870,15.975117,8.560527,4.248939,2.432146,2.700563,4.332023,...,4.914675,6.751660,6.020052,3.641826,4.516870,10.352589,11.004499,9.246391,8.224123,10.247271
9,F0000047,1.926024,2.275108,2.933197,3.494335,2.513233,1.539907,1.563043,1.575975,2.180220,...,5.074250,6.098646,4.938331,3.050952,4.389782,5.872658,5.134856,6.942337,10.453601,8.602502


In [0]:
!cp drive/aoi_test/t_brain/geo/test_submit.csv .

In [0]:
from google.colab import files
files.download('test_submit.csv')